The objective of this notebook is to preprocess the dataset 

## Libraries

In [ ]:
import pandas as pd 
import re
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer


In [4]:
df = pd.read_csv("../data/bronze/spam.csv")


### Change data types

In [5]:
df['Category'] = df['Category'].map({"ham": 0, "spam": 1}).astype(int)
df['Message'] = df['Message'].astype(str)

In [ ]:
df_duplicated = df['Message'].duplicated().sum()

Total duplicated rows: 415


In [8]:
df.drop_duplicates(inplace=True)

In [19]:
def clean_text(text):
    special_replacements = {
        r"£": "pound",
        r"\$": "dollar",
        r"\€": "euro",
        r"%": "percentage", 
        r"ì": "i",
        r"ü": "you",
        }
    
    emoticon_pattern = re.compile(r"""
    [:;=Xx]           
    [-~]?             
    [\)\]\(\[dDpP/]   
    """, re.VERBOSE)
    
    for pattern, replacement in special_replacements.items():
        text = re.sub(pattern, replacement, text)
    text = re.sub(emoticon_pattern, 'emoji', text)
    text = text.lower()
    text = re.sub('<[^<>]+>', ' ', text)
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub('[0-9]+', 'number', text)
    text = re.sub('[^\s]+@[^\s]+', 'emailaddr', text)
    text = text.translate(str.maketrans('', '', punctuation))
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
df['Message']=df['Message'].apply(clean_text)
df['Message'] = df['Message'].apply(word_tokenize)
df['Message'] = df['Message'].apply(lambda x: [word for word in x if word not in stop_words])

lemmatizer=WordNetLemmatizer()
def lem_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['Message'] = df['Message'].apply(lem_tokens)
df = df[df['Message'].str.strip().astype(bool)]

In [ ]:
df.to_csv("../data/silver/df_preprocessed.csv", index= False)